In [4]:
from cluster.cluster import make_clusters
from input.grid import grid as Grid
from graph.make_graph import make_graph
from graph.find_path import find_optimal_path
from random_state_generator.random_state_generator import random_state_generator
import time
import numpy as np
import pandas as pd

def run_experiment(length,height,num):
    
    k = 2 #how much we want to shorten the grid into; for example for k=2 100x100 turns into 50x50; for k=5 100x100 turns into 20x20
    rectangle,source,dest = random_state_generator(length,height,num,gap=k+3)
    grid = Grid(rectangle)

    start = time.time()
    clusters = make_clusters(rectangle)
    graph = make_graph(clusters,source,dest)
    grid.add_graph(graph)

    # optimal path

    opt_path,opt_path_len = find_optimal_path(graph,source,dest)
    end = time.time()
    time_taken = end-start
    # print(f"time taken :{end-start}, path length: {opt_path_len}")


    # meta


    grid.make_meta_grid(k=k)
    start = time.time()
    meta_clusters = make_clusters(grid.meta_rect)
    meta_src = (grid.meta_rect.shape[0]-1,0)
    meta_dest = (0,grid.meta_rect.shape[1]-1)
    meta_graph = make_graph(meta_clusters,meta_src,meta_dest)
    grid.add_graph(meta_graph,type="meta")


    # reverse mapping
    met_path_graph = grid.print_reverse_meta_grid(meta_graph,k,print=False)
    src = (meta_src[0]*k,meta_src[1]*k)
    dst = (meta_dest[0]*k,meta_dest[1]*k)
    meta_opt_path,meta_opt_path_len = find_optimal_path(met_path_graph,src,dst)
    end = time.time()
    time_taken_rev = end - start
    # print(f"time taken :{end-start}, path length: {meta_opt_path_len}")
    
    return opt_path_len,meta_opt_path_len,time_taken,time_taken_rev

param_list = [[40,30,5],[300,400,6],[4000,5000,35],[5000,4200,25],[6000,5000,30]]
# param_list = [[40,30,5],[300,400,6]]

arr = np.array(param_list)
opl_list = []
mopl_list = []

inc_len_list = []

tt_list = []
ttr_list = []

decr_t_list = []



for l,h,n in param_list:
    a,b,c,d = run_experiment(l,h,n)
    opl_list.append(a)
    mopl_list.append(b)
    tt_list.append(c)
    ttr_list.append(d)
    inc_l,dec_t = None,None
    if a>0:
        inc_l = ((b-a)/a)*100
    if c>0:
        dec_t = ((c-d)/c)*100
    inc_len_list.append(inc_l)
    decr_t_list.append(dec_t)
    
arr = np.concatenate((arr,np.array([opl_list]).T),axis=1)
arr = np.concatenate((arr,np.array([mopl_list]).T),axis=1)
arr = np.concatenate((arr,np.array([tt_list]).T),axis=1)
arr = np.concatenate((arr,np.array([ttr_list]).T),axis=1)
arr = np.concatenate((arr,np.array([inc_len_list]).T),axis=1)
arr = np.concatenate((arr,np.array([decr_t_list]).T),axis=1)

# print(arr)

column_names = ["l","h","n","path_len","path_len_1","time","time_1","%l inc","%t decr"]
df = pd.DataFrame(arr,columns=column_names)
df

,l,h,n,path_len,path_len_1,time,time_1,%l inc,%t decr
0,40.0,30.0,5.0,51.630967,55.698830,0.006340,0.000000,7.878728,100.000000
1,300.0,400.0,6.0,533.816913,537.457779,0.162037,0.032634,0.682044,79.860248
2,4000.0,5000.0,35.0,8587.224468,8589.783612,18.845038,4.376650,0.029802,76.775584
3,5000.0,4200.0,25.0,7713.704394,7716.994218,17.852111,4.451523,0.042649,75.064445
4,6000.0,5000.0,30.0,9553.485828,9556.771521,25.937072,6.452191,0.034393,75.123674
